# Prerequisite

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import wfdb # read Physionet/picsdb file format
from heartrate import *
import pandas as pd
import re
from scipy.signal import medfilt, resample
from scipy.interpolate import interp1d


# Data Analysis

In [ ]:
#A
segments = {}
# for i in range(10):
#     key = f"infant{i+1:d}"
#     segments[key] = []
# segments['infant1'] = [(3550, 3750), (4720,4822), (5100,5280), (7760,7870), (11435,11550)]
# segments['infant2'] = [(3990,4090), (9440,9580), (9770,9890), (11940,12080), (12180,12350)]
# segments['infant3'] = [(5575,5760), (10200,10340), (10400,10600), (13300,13500), (13700,13940)]
segments['infant4'] = [(5860,6020), (6695,6810), (6900,7020), (8350,8450), (12750,12980)]
segments['infant5'] = [(103750,104000), (121275, 121375), (122475,122575), (146500,146600), (147050,147350)]
segments['infant6'] = [(7130,7230), (7350,7530), (11440,11600), (13290,13390), (13530,13670)]
segments['infant7'] = [(5060,5190), (5480,5590), (9150,9450), (9500,9675), (13420,13630)]
segments['infant8'] = [(4880,5000), (5040,5140), (6690,6790), (7500,7700), (14440,14570)]
segments['infant9'] = [(5940,6040), (7620,7740), (11120,11220), (11540,11640), (12400,12580)]
segments['infant10'] = [(3225,3400), (9500,9700), (10150,10350), (14500,14600), (14610,14710)]


# Data Preprocessing

In [11]:
base_dir = os.getcwd()
data_dir = "C:/um/machine learning/group assignment/machine-learning-group/data"
print("base directory: ", base_dir)
print("data directory: ", data_dir)

export_folder = os.path.join(base_dir,"exports")
print(f"export folder: {export_folder:s}")
if not os.path.isdir(export_folder):
    os.mkdir(export_folder)
    print("export folder created...")
else:
    print("export folder already exists...")

base directory:  c:\um\machine learning\group assignment\machine-learning-group
data directory:  C:/um/machine learning/group assignment/machine-learning-group/data
export folder: c:\um\machine learning\group assignment\machine-learning-group\exports
export folder already exists...


In [2]:
xls_file="./input_settings2.xlsx"
segments = load_segments(xls_file, verbose=False)
# segments = prepare_segments(verbose=False)
# segments['infant1']['segment0']['freq_hi_ecg'] = 120
# segments['infant5']['segment0']['freq_hi_ecg'] = 120

In [3]:
# Verify data
for key_subj in segments:
    for key_seg in segments[key_subj]:
        pass
        print(key_subj, key_seg, "\n", segments[key_subj][key_seg])

infant4 segment0 
 {'on': 5860, 'off': 6020, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 125.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.2, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant4 segment1 
 {'on': 6695, 'off': 6810, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 125.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.2, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant4 segment2 
 {'on': 6900, 'off': 7020, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 125.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.2, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant4 segment3 
 {'on': 8350, 'off': 8450, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 125.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.2, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant4 segment4 
 {'on': 12750, 'off': 12980, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 125.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.2, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_

In [4]:
def preprocess_data(key_subj, key_seg):
    file_index = int(re.findall(r'infant(\d+)', key_subj)[0])
    segment_index = int(re.findall(r'segment(\d+)', key_seg)[0])
    print(f"Processing: {key_subj:s} {key_seg:s}")
    print(f"file_index: {file_index:d}\nsegment_index: {segment_index:d}")
    file_ecg = f"infant{file_index:d}_ecg"
    file_resp = f"infant{file_index:d}_resp"
    print("Loading ECG file: ", file_ecg)
    print("Loading RESP file: ", file_resp)
    # data import
    x_ecg_full, x_resp_full, fs_ecg, fs_resp = load_waveforms(data_dir, file_index)
    
    # Sampling frequencies [Hz] and sampling intervals [seconds]:
    dt_ecg = 1/fs_ecg # ECG sampling interval in sec.
    dt_resp = 1/fs_resp # RESP sampling interval in sec.
    #print("ECG sampling interval dt = ", dt_ecg, " sec.")
    #print("RESP sampling interval dt = ", dt_resp, " sec.")
    
    # get segment borders in seconds from the 'segments' dictionary
    t0_sec = segments[key_subj][key_seg]['on']
    t1_sec = segments[key_subj][key_seg]['off']
    # full time
    # t1_sec = x_ecg_full.shape[0]*dt_ecg 
    
    # convert seconds to samples, respect different sampling rates for ECG and RESP signals
    t0_sample_ecg = round(t0_sec * fs_ecg)
    t1_sample_ecg = round(t1_sec * fs_ecg)
    t0_sample_resp = round(t0_sec * fs_resp)
    t1_sample_resp = round(t1_sec * fs_resp)
    
    # Extract the ECG and RESP data arrays and convert them to a 1-dimensional arrays:
    x_ecg = x_ecg_full[t0_sample_ecg:t1_sample_ecg]
    x_resp = x_resp_full[t0_sample_resp:t1_sample_resp]
    
    #''' Invert ECG signal if necessary
    if (key_subj=='infant10'):
        #mn = np.min(x_ecg-np.median(x_ecg))
        #mx = np.max(x_ecg-np.median(x_ecg))
        #if np.abs(mn) > np.abs(mx):
        print("Invert ECG signal!")
        x_ecg = -x_ecg
    #'''
    
    # time axes
    time_ecg = np.arange(x_ecg.shape[0])*dt_ecg # ECG time axis
    time_resp = np.arange(x_resp.shape[0])*dt_resp # RESP time axis
    
    # cut-off frequencies for band-pass filtering
    freq_lo_ecg = segments[key_subj][key_seg]['freq_lo_ecg']
    freq_hi_ecg = segments[key_subj][key_seg]['freq_hi_ecg']
    freq_lo_resp = segments[key_subj][key_seg]['freq_lo_resp']
    freq_hi_resp = segments[key_subj][key_seg]['freq_hi_resp']
    
    # apply the band-pass filter
    x_ecg_filt  = bp_filter(x_ecg, fs_ecg, freq_lo_ecg, freq_hi_ecg)
    x_resp_filt = bp_filter(x_resp, fs_resp, freq_lo_resp, freq_hi_resp)
    
    # get thresholds
    thr_ecg = segments[key_subj][key_seg]['thr_ecg']
    thr_resp = segments[key_subj][key_seg]['thr_resp']
    # local maxima (1st round)
    locmax_ecg = locmax(x_ecg_filt)
    locmax_resp = locmax(x_resp_filt)
    w = 1
    ecg_peaks = np.array([np.mean(x_ecg_filt[i-w:i+w]) for i in locmax_ecg])
    resp_peaks = np.array([np.mean(x_resp_filt[i-w:i+w]) for i in locmax_resp])
    # apply thresholds
    locmax_ecg = np.array([j for i, j in enumerate(locmax_ecg) if ecg_peaks[i] > thr_ecg])
    locmax_resp = np.array([j for i, j in enumerate(locmax_resp) if resp_peaks[i] > thr_resp])
    
    # remove ECG peaks too close together
    f_max_ecg = segments[key_subj][key_seg]['f_max_ecg']
    pp_min_ecg = 1/f_max_ecg # sec.
    n_min_ecg = np.round(pp_min_ecg/dt_ecg)
    locmax_ecg_copy = locmax_ecg.copy()
    for i in range(len(locmax_ecg)-1):
        if (locmax_ecg[i+1]-locmax_ecg[i] < n_min_ecg):
            locmax_ecg_copy[i+1] = 0
    locmax_ecg = locmax_ecg_copy
    locmax_ecg = locmax_ecg[locmax_ecg>0]
    
    # remove RESP peaks too close together
    f_max_resp = segments[key_subj][key_seg]['f_max_resp']
    pp_min_resp = 1/f_max_resp # sec.
    n_min_resp = np.round(pp_min_resp/dt_resp)
    locmax_resp_copy = locmax_resp.copy()
    for i in range(len(locmax_resp)-1):
        if (locmax_resp[i+1]-locmax_resp[i] < n_min_resp):
            locmax_resp_copy[i+1] = 0
    locmax_resp = locmax_resp_copy
    locmax_resp = locmax_resp[locmax_resp>0]
    
    # interval statistics
    print("\nECG stats:")
    ecg_intervals = interval_stats(locmax_ecg, dt_ecg)
    # print(ecg_intervals)
    
    print("\nRESP stats:")
    resp_intervals = interval_stats(locmax_resp, dt_resp)
    # print(resp_intervals)
    
    # save
    if (key_subj=='infant10'):
        x_ecg = -x_ecg
        x_ecg_filt = -x_ecg_filt
        
    file_time_ecg = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_time.txt")
    file_time_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_time.txt")
    np.savetxt(file_time_ecg, time_ecg, fmt='%.5f', delimiter='\n')
    np.savetxt(file_time_resp, time_resp, fmt='%.5f', delimiter='\n')
    
    file_ecg  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg.txt")
    file_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp.txt")
    np.savetxt(file_ecg, x_ecg, fmt='%.5f', delimiter='\n')
    np.savetxt(file_resp, x_resp, fmt='%.5f', delimiter='\n')
    
    file_ecg_filt  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_filt.txt")
    file_resp_filt = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_filt.txt")
    np.savetxt(file_ecg_filt, x_ecg_filt, fmt='%.5f', delimiter='\n')
    np.savetxt(file_resp_filt, x_resp_filt, fmt='%.5f', delimiter='\n')
    
    file_locmax_ecg  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_locmax_ecg.txt")
    file_locmax_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_locmax_resp.txt")
    np.savetxt(file_locmax_ecg, locmax_ecg, fmt='%.5f', delimiter='\n')
    np.savetxt(file_locmax_resp, locmax_resp, fmt='%.5f', delimiter='\n')
    
    file_ecg_intervals  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_intervals.txt")
    file_resp_intervals = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_intervals.txt")
    np.savetxt(file_ecg_intervals, ecg_intervals, fmt='%.5f', delimiter='\n')
    np.savetxt(file_resp_intervals, resp_intervals, fmt='%.5f', delimiter='\n')

In [5]:
for key_subj in segments:
    for key_seg in segments[key_subj]:
        preprocess_data(key_subj, key_seg)
print("Completed Preprocessing")

Processing: infant4 segment0
file_index: 4
segment_index: 0
Loading ECG file:  infant4_ecg
Loading RESP file:  infant4_resp



KeyboardInterrupt



In [8]:
def generateHeartRate(key_subj, key_seg):
    # key_subj = f"infant{file_index}"
    # key_seg = f"segment{segment_index}"

    # load exported data
    file_time_ecg = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_time.txt")
    file_time_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_time.txt")
    time_ecg = np.loadtxt(file_time_ecg)
    time_resp = np.loadtxt(file_time_resp)

    file_ecg  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg.txt")
    file_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp.txt")
    x_ecg = np.loadtxt(file_ecg)
    x_resp = np.loadtxt(file_resp)

    file_ecg_filt  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_filt.txt")
    file_resp_filt = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_filt.txt")
    x_ecg_filt = np.loadtxt(file_ecg_filt)
    x_resp_filt = np.loadtxt(file_resp_filt)

    file_locmax_ecg  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_locmax_ecg.txt")
    file_locmax_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_locmax_resp.txt")
    locmax_ecg = np.loadtxt(file_locmax_ecg).astype('int')
    locmax_resp = np.loadtxt(file_locmax_resp).astype('int')

    file_ecg_intervals  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_intervals.txt")
    file_resp_intervals = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_intervals.txt")
    ecg_intervals = np.loadtxt(file_ecg_intervals)
    resp_intervals = np.loadtxt(file_resp_intervals)

    dt_ecg = time_ecg[1]-time_ecg[0]
    dt_resp = time_resp[1]-time_resp[0]
    fs_ecg = 1/dt_ecg # ECG sampling rate in Hz
    fs_resp = 1/dt_resp # RESP sampling rate in Hz
    print("ECG sampling frequency: ", fs_ecg, " Hz")
    print("ECG sampling interval dt = ", dt_ecg, " sec.")
    print("RESP sampling frequency: ", fs_resp, " Hz")
    print("RESP sampling interval dt = ", dt_resp, " sec.")

    # interpolate ECG and RESP peak-to-peak intervals
    t_mid_ecg = dt_ecg*(locmax_ecg[1:] + locmax_ecg[:-1])/2.0
    t_mid_resp = dt_resp*(locmax_resp[1:] + locmax_resp[:-1])/2.0
    #ecg_int_mid = (ecg_intervals[1:] - ecg_intervals[:-1])/2.0
    #resp_int_mid = (resp_intervals[1:] - resp_intervals[:-1])/2.0
    ecg_int_mid = 60./ecg_intervals
    resp_int_mid = 60./resp_intervals

    # avoid edge effects, insert first and last time stamps
    t_mid_ecg = np.insert(t_mid_ecg, 0, time_ecg[0])
    t_mid_ecg = np.insert(t_mid_ecg, t_mid_ecg.shape[0], time_ecg[-1])
    t_mid_resp = np.insert(t_mid_resp, 0, time_resp[0])
    t_mid_resp = np.insert(t_mid_resp, t_mid_resp.shape[0], time_resp[-1])
    # repeat first and last ECG/RESP frequency
    ecg_int_mid = np.insert(ecg_int_mid, 0, ecg_int_mid[0])
    ecg_int_mid = np.insert(ecg_int_mid, ecg_int_mid.shape[0], ecg_int_mid[-1])
    resp_int_mid = np.insert(resp_int_mid, 0, resp_int_mid[0])
    resp_int_mid = np.insert(resp_int_mid, resp_int_mid.shape[0], resp_int_mid[-1])

    #fig, ax = plt.subplots(2, 1, figsize=(16,6))
    #ax[0].plot(t_mid_ecg, ecg_intervals, '-ok', lw=2)
    #ax[1].plot(t_mid_resp, resp_intervals, '-ok', lw=2)
    #ax[0].plot(ecg_intervals, '-ok', lw=2)
    #ax[1].plot(resp_intervals, '-ok', lw=2)
    #plt.show()

    #print(t_mid_ecg.shape, t_mid_resp.shape, ecg_int_mid.shape, resp_int_mid.shape)

    ipmode='cubic'  # 'linear'
    fip_ecg = interp1d(t_mid_ecg, ecg_int_mid, kind=ipmode, fill_value='extrapolate')
    fip_resp = interp1d(t_mid_resp, resp_int_mid, kind=ipmode, fill_value='extrapolate')
    ecg_rate = fip_ecg(time_ecg)
    resp_rate = fip_resp(time_resp)
    ksize = 3
    ecg_rate = medfilt(ecg_rate,kernel_size=ksize)
    resp_rate = medfilt(resp_rate,kernel_size=ksize)
    #print(ecg_rate)

    # add norm boundaries, values from: [REF.]
    ecg_rate_norm_lo = 100
    ecg_rate_norm_hi = 160
    resp_rate_norm_lo = 40
    resp_rate_norm_hi = 60
    # percentages outside the norm ranges
    pct_ecg_ltref = 100.*len(np.where(ecg_rate < ecg_rate_norm_lo)[0])/len(ecg_rate)
    pct_ecg_gtref = 100.*len(np.where(ecg_rate > ecg_rate_norm_hi)[0])/len(ecg_rate)
    pct_resp_ltref = 100.*len(np.where(resp_rate < resp_rate_norm_lo)[0])/len(resp_rate)
    pct_resp_gtref = 100.*len(np.where(resp_rate > resp_rate_norm_hi)[0])/len(resp_rate)
    print(f"Percentage of ECG rate below {ecg_rate_norm_lo:.1f} bpm: {pct_ecg_ltref:.1f} %")
    print(f"Percentage of ECG rate above {ecg_rate_norm_hi:.1f} bpm: {pct_ecg_gtref:.1f} %")
    print(f"Percentage of RESP rate below {resp_rate_norm_lo:.1f} per min.: {pct_resp_ltref:.1f} %")
    print(f"Percentage of RESP rate above {resp_rate_norm_hi:.1f} per min.: {pct_resp_gtref:.1f} %")


    pd.DataFrame({'heart_rate': resample(ecg_rate, int(len(ecg_rate)/fs_ecg * 50)), 
                'respiratory data': resample(x_resp, int(len(x_resp)/fs_resp * 50)),
                'filtered respiratory data': resample(x_resp_filt, int(len(x_resp_filt)/fs_resp * 50)),
                'respiratory rate': resample(resp_rate, int(len(resp_rate)/fs_resp * 50))
                    }).to_csv(f'processed_{key_subj}_{key_seg}.csv', index=False)



In [14]:
for key_subj in segments:
    for key_seg in segments[key_subj]:
        print(f"processing {key_subj} {key_seg}")
        generateHeartRate(key_subj, key_seg)


processing infant4 segment0
ECG sampling frequency:  500.0  Hz
ECG sampling interval dt =  0.002  sec.
RESP sampling frequency:  50.0  Hz
RESP sampling interval dt =  0.02  sec.
Percentage of ECG rate below 100.0 bpm: 0.0 %
Percentage of ECG rate above 160.0 bpm: 62.4 %
Percentage of RESP rate below 40.0 per min.: 2.5 %
Percentage of RESP rate above 60.0 per min.: 69.4 %
processing infant4 segment1
ECG sampling frequency:  500.0  Hz
ECG sampling interval dt =  0.002  sec.
RESP sampling frequency:  50.0  Hz
RESP sampling interval dt =  0.02  sec.
Percentage of ECG rate below 100.0 bpm: 0.0 %
Percentage of ECG rate above 160.0 bpm: 7.5 %
Percentage of RESP rate below 40.0 per min.: 0.6 %
Percentage of RESP rate above 60.0 per min.: 92.5 %
processing infant4 segment2
ECG sampling frequency:  500.0  Hz
ECG sampling interval dt =  0.002  sec.
RESP sampling frequency:  50.0  Hz
RESP sampling interval dt =  0.02  sec.
Percentage of ECG rate below 100.0 bpm: 0.0 %
Percentage of ECG rate above 

FileNotFoundError: c:\um\machine learning\group assignment\machine-learning-group\exports\infant6_segment0_ecg_time.txt not found.